In [1]:
import os
import random
import urllib

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from PIL import Image
from torchvision import transforms

import torch
import torchvision
from torchvision import datasets, transforms

from customDataset import ISICDataset

from data_exploration_helper import dataset_overview

from data_augmentation import augment_images_and_save_to_file

In [2]:
#########################################################
# ==================== CONSTANTS ========================
#########################################################

TRAIN_2018_LABELS: str = "./data/ISIC2018_Training_GroundTruth.csv"
TRAIN_2018_ROOT_DIR: str = "./data/ISIC2018_Training_Input"

NEW_TRAIN_2018_ROOT_DIR: str = "./augmented_data/ISIC2018_Training_Input"
NEW_TRAIN_2018_LABELS: str = "./augmented_data/ISIC2018_Training_GroundTruth.csv"


IMAGE_FILE_TYPE: str = "jpg"
TRAIN_NROWS: int = None # SET TO None if you want all samples
TEST_NROWS: int = None # SET TO None if you want all samples

In [3]:
# Define image pre-processing steps
# Define the transforms to apply to the training data
augmentation_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.RandAugment(num_ops=4, magnitude=2, num_magnitude_bins=10),
    transforms.ToPILImage()
])

pil_image_transform = transforms.Compose([
    transforms.ToPILImage(), # Removes error
])

In [4]:
augment_images_and_save_to_file(TRAIN_2018_ROOT_DIR, NEW_TRAIN_2018_ROOT_DIR, TRAIN_2018_LABELS, NEW_TRAIN_2018_LABELS, augmentation_transform, min_number_of_each_class=10)

 36%|███▋      | 405/1113 [00:14<00:25, 27.38it/s]


KeyboardInterrupt: 